In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# =============================================================================
# 1. Navier–Stokes PDE solver (pseudo-spectral method)
# =============================================================================
def solve_navier_stokes(w0, nu, dt, t_final, N):
    """
    2D Navier–Stokes vorticity PDE (주기적 경계조건) 를 푸는 간단한 pseudo-spectral solver.
    
    PDE:
        ∂ₜw + u · ∇w = ν ∆w + f(x)
        u = (ψ_y, -ψ_x),   and   ∆ψ = -w,  with periodic boundary conditions.
    입력:
        w0: 초기 vorticity, shape (N, N)
        nu: 점성 계수
        dt: 시간 스텝
        t_final: 최종 시간
        N: 격자 크기 (N×N)
    출력:
        snapshots: 정수 시간마다 (t, w) 튜플 리스트.
    """
    # 공간 좌표 생성 (0 ≤ x,y < 1)
    x = np.linspace(0, 1, N, endpoint=False)
    y = np.linspace(0, 1, N, endpoint=False)
    xv, yv = np.meshgrid(x, y, indexing='ij')
    
    # forcing function: f(x) = 0.1*( sin(2π(x+y)) + cos(2π(x+y)) )
    f = 0.1 * (np.sin(2*np.pi*(xv + yv)) + np.cos(2*np.pi*(xv + yv)))
    
    # FFT frequencies (주기적 도메인: [0,1])
    kx = np.fft.fftfreq(N, d=1.0/N) * 2*np.pi
    ky = np.fft.fftfreq(N, d=1.0/N) * 2*np.pi
    kx, ky = np.meshgrid(kx, ky, indexing='ij')
    ksq = kx**2 + ky**2
    ksq[0,0] = 1.0  # 0분할 방지
    
    w = w0.copy()
    snapshots = []
    snapshot_times = np.arange(0, t_final+dt, 1.0)  # 정수 시간마다 기록
    next_snapshot_index = 0
    
    nsteps = int(t_final/dt)
    for step in range(nsteps):
        t = step * dt
        # 스냅샷 기록
        if next_snapshot_index < len(snapshot_times) and t >= snapshot_times[next_snapshot_index]:
            snapshots.append((t, w.copy()))
            next_snapshot_index += 1
        
        # vorticity의 FFT
        w_hat = np.fft.fft2(w)
        # stream function: ∆ψ = -w  ->  ψ_hat = -w_hat / ksq, ψ_hat(0)=0
        psi_hat = -w_hat / ksq
        psi_hat[0,0] = 0.0
        psi = np.real(np.fft.ifft2(psi_hat))
        
        # 속도 계산: u = ψ_y, v = -ψ_x (spectral differentiation)
        u = np.real(np.fft.ifft2(1j * ky * psi_hat))
        v = -np.real(np.fft.ifft2(1j * kx * psi_hat))
        
        # vorticity gradient (spectral differentiation)
        w_x = np.real(np.fft.ifft2(1j * kx * w_hat))
        w_y = np.real(np.fft.ifft2(1j * ky * w_hat))
        
        # 비선형 항: u * w_x + v * w_y
        nonlinear = u * w_x + v * w_y
        
        # Laplacian: ∆w = ifft2(-ksq * w_hat)
        lap_w = np.real(np.fft.ifft2(-ksq * w_hat))
        
        # Euler explicit time stepping
        w = w + dt * (-nonlinear + nu * lap_w + f)
    
    return snapshots



In [2]:

# =============================================================================
# 2. Navier-Stokes 데이터셋 (PDE solver를 사용하여 데이터 생성)
# =============================================================================
class NavierStokesDatasetFromSolver(Dataset):
    def __init__(self, num_samples, N, dt, t_final, nu, T_in_range, T_out_times):
        """
        num_samples: 샘플 개수.
        N: 공간 격자 크기 (N×N).
        dt: 시간 스텝.
        t_final: 최종 시각 (예, 30).
        nu: 점성 계수.
        T_in_range: 입력 시간 구간 (예: (0, 10)).
        T_out_times: 타겟 시간 (예: [15, 20, 25, 30]).
        """
        self.samples = []
        for i in range(num_samples):
            # 랜덤 초기 조건 (예: 가우시안 랜덤 필드)
            w0 = np.random.randn(N, N)
            snapshots = solve_navier_stokes(w0, nu, dt, t_final, N)
            # snapshots: [(t, w), ...] t는 정수 시간 (0,1,2,...,t_final)
            input_series = []
            for t, w in snapshots:
                if t >= T_in_range[0] and t <= T_in_range[1]:
                    input_series.append(w)
            input_series = np.array(input_series)  # shape: (T_in, N, N)
            
            target_series = []
            for t, w in snapshots:
                if t in T_out_times:
                    target_series.append(w)
            target_series = np.array(target_series)  # shape: (T_out, N, N)
            self.samples.append((input_series, target_series))
        
        # 공간 좌표 (x, y) 생성, x,y ∈ [0,1)
        x = np.linspace(0, 1, N, endpoint=False)
        y = np.linspace(0, 1, N, endpoint=False)
        xv, yv = np.meshgrid(x, y, indexing='ij')
        self.spatial_coords = np.stack([xv, yv], axis=-1)  # (N, N, 2)
        # 입력 시간 좌표: 정수 시간 (예: 0,1,...,10)
        self.input_times = np.arange(T_in_range[0], T_in_range[1]+1)
        self.target_times = np.array(T_out_times)
        
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        input_series, target_series = self.samples[idx]
        T_in = input_series.shape[0]
        N = input_series.shape[1]
        # 각 입력 시간(t)에 대해 [vorticity, t, x, y]를 결합
        input_data = []
        for i, t in enumerate(self.input_times):
            t_field = np.full((N, N, 1), t)
            w_field = input_series[i][:, :, np.newaxis]
            combined = np.concatenate([w_field, t_field, self.spatial_coords], axis=-1)
            input_data.append(combined)
        input_data = np.stack(input_data, axis=0)  # shape: (T_in, N, N, 4)
        
        # target_series의 길이가 기대하는 T_out와 같지 않은 경우, 부족한 부분을 마지막 스냅샷으로 패딩
        if target_series.shape[0] < len(self.target_times):
            pad_len = len(self.target_times) - target_series.shape[0]
            last = target_series[-1:,...]
            pad = np.repeat(last, pad_len, axis=0)
            target_series = np.concatenate([target_series, pad], axis=0)
        
        target_data = []
        for i in range(target_series.shape[0]):
            target_data.append(target_series[i][:, :, np.newaxis])
        target_data = np.stack(target_data, axis=0)  # shape: (T_out, N, N, 1)
        
        # Torch tensor로 변환
        input_data = torch.tensor(input_data, dtype=torch.float32)
        target_data = torch.tensor(target_data, dtype=torch.float32)
        return input_data, target_data


In [ ]:

# =============================================================================
# 3. FNO 모델 (3D FNO: 시간-공간 연산자 학습) with Physics-Informed Loss
# =============================================================================
class SpectralConv3d(nn.Module):
    def __init__(self, in_channels, out_channels, modes_t, modes_x, modes_y):
        super(SpectralConv3d, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes_t = modes_t
        self.modes_x = modes_x
        self.modes_y = modes_y
        self.scale = 1 / (in_channels * out_channels)
        self.weights = nn.Parameter(
            self.scale * torch.rand(in_channels, out_channels, modes_t, modes_x, modes_y, 2)
        )
    
    def compl_mul3d(self, input, weights):
        # weights: (in_channels, out_channels, t, x, y, 2) -> 복소수 텐서: (in_channels, out_channels, t, x, y)
        weight_complex = torch.view_as_complex(weights)
        # "bctxy, cotxy -> botxy": 입력의 각 주파수에 대해, C_in에 대해 합산하여 (batch, out_channels, t, x, y)를 얻음.
        return torch.einsum("bctxy, cotxy -> botxy", input, weight_complex)
    
    def forward(self, x):
        batchsize = x.shape[0]
        # 3D FFT 계산: x의 shape은 (batch, in_channels, t, x, y)
        x_ft = torch.fft.rfftn(x, dim=[2,3,4])
        out_ft = torch.zeros(
            batchsize, self.out_channels, x_ft.size(2), x_ft.size(3), x_ft.size(4),
            device=x.device, dtype=torch.cfloat
        )
        t_modes = min(self.modes_t, x_ft.size(2))
        x_modes = min(self.modes_x, x_ft.size(3))
        y_modes = min(self.modes_y, x_ft.size(4))
        out_ft[:, :, :t_modes, :x_modes, :y_modes] = self.compl_mul3d(
            x_ft[:, :, :t_modes, :x_modes, :y_modes],
            self.weights[:, :, :t_modes, :x_modes, :y_modes]
        )
        x = torch.fft.irfftn(out_ft, s=(x.size(2), x.size(3), x.size(4)))
        return x


class FNO3d(nn.Module):
    def __init__(self, modes_t, modes_x, modes_y, width, T_in, T_out):
        """
        modes_t, modes_x, modes_y: 시간 및 공간 방향의 Fourier 모드 수.
        width: latent channel 차원.
        T_in: 입력 시간 스텝 수 (예: 11, t=0~10)
        T_out: 출력 시간 스텝 수 (예: 4, t=15,20,25,30)
        """
        super(FNO3d, self).__init__()
        self.width = width
        self.T_in = T_in
        self.T_out = T_out
        self.fc0 = nn.Linear(4, width)  # 입력 채널: [vorticity, t, x, y]
        
        self.conv0 = SpectralConv3d(self.width, self.width, modes_t, modes_x, modes_y)
        self.conv1 = SpectralConv3d(self.width, self.width, modes_t, modes_x, modes_y)
        self.conv2 = SpectralConv3d(self.width, self.width, modes_t, modes_x, modes_y)
        self.conv3 = SpectralConv3d(self.width, self.width, modes_t, modes_x, modes_y)
        
        self.w0 = nn.Conv3d(self.width, self.width, 1)
        self.w1 = nn.Conv3d(self.width, self.width, 1)
        self.w2 = nn.Conv3d(self.width, self.width, 1)
        self.w3 = nn.Conv3d(self.width, self.width, 1)
        
        self.fc1 = nn.Linear(self.width, 128)
        self.fc2 = nn.Linear(128, 1)  # 출력: vorticity
        
        # Temporal predictor: 각 픽셀별로 입력 시간(T_in)을 출력 시간(T_out)으로 매핑
        self.temporal_fc = nn.Linear(T_in, T_out)
        
    def forward(self, x):
        # x: (batch, T_in, H, W, 4)
        batch, T_in, H, W, _ = x.shape
        x = self.fc0(x)  # (batch, T_in, H, W, width)
        x = x.permute(0, 4, 1, 2, 3)  # (batch, width, T_in, H, W)
        
        x0 = self.conv0(x)
        x1 = self.w0(x)
        x = F.gelu(x0 + x1)
        
        x0 = self.conv1(x)
        x1 = self.w1(x)
        x = F.gelu(x0 + x1)
        
        x0 = self.conv2(x)
        x1 = self.w2(x)
        x = F.gelu(x0 + x1)
        
        x0 = self.conv3(x)
        x1 = self.w3(x)
        x = F.gelu(x0 + x1)
        
        x = x.permute(0, 2, 3, 4, 1)  # (batch, T_in, H, W, width)
        x = self.fc1(x)
        x = F.gelu(x)
        x = self.fc2(x)  # (batch, T_in, H, W, 1)
        
        # Temporal prediction: 각 픽셀별로 입력 시간 차원(T_in)을 T_out 차원으로 변환
        x = x.squeeze(-1)              # (batch, T_in, H, W)
        x = x.permute(0, 2, 3, 1)        # (batch, H, W, T_in)
        x = self.temporal_fc(x)          # (batch, H, W, T_out)
        x = x.permute(0, 3, 1, 2).unsqueeze(-1)  # (batch, T_out, H, W, 1)
        return x

# =============================================================================
# Physics-Informed Loss 관련 함수
# =============================================================================
def compute_velocity_torch(w):
    """
    w: (batch, H, W)
    FFT를 이용해 stream function ψ를 구한 뒤, u = ψ_y, v = -ψ_x를 계산합니다.
    """
    batch, H, W = w.shape
    device = w.device
    kx = torch.fft.fftfreq(H, d=1.0/H).to(device) * 2 * np.pi
    ky = torch.fft.fftfreq(W, d=1.0/W).to(device) * 2 * np.pi
    kx, ky = torch.meshgrid(kx, ky, indexing='ij')
    kx = kx.unsqueeze(0)  # (1, H, W)
    ky = ky.unsqueeze(0)  # (1, H, W)
    ksq = kx**2 + ky**2
    # 0번 주파수에서 0으로 나누는 것을 피하기 위해
    ksq[0,0,0] = 1.0

    w_hat = torch.fft.fft2(w)  # (batch, H, W)
    psi_hat = -w_hat / ksq
    psi_hat[:,0,0] = 0.0
    psi = torch.real(torch.fft.ifft2(psi_hat))
    u_hat = 1j * ky * psi_hat
    v_hat = -1j * kx * psi_hat
    u = torch.real(torch.fft.ifft2(u_hat))
    v = torch.real(torch.fft.ifft2(v_hat))
    return u, v

def compute_physics_loss(pred, nu, dt, dx, dy):
    """
    pred: (batch, T_out, H, W, 1) 예측된 vorticity
    nu: 점성 계수
    dt: 시간 간격 (출력 시간 간격)
    dx, dy: 공간 간격 (도메인이 [0,1]이므로 dx=1/W, dy=1/H)
    
    Navier-Stokes vorticity PDE:
        ∂ₜw + u·∇w = ν Δw + f(x)
    f(x) = 0.1*(sin(2π(x+y)) + cos(2π(x+y)))
    
    finite difference를 이용해 각 항을 근사하고, PDE 잔차의 MSE를 계산합니다.
    """
    batch, T_out, H, W, _ = pred.shape
    pred = pred.squeeze(-1)  # (batch, T_out, H, W)
    
    # spatial grid (동일한 for 모든 배치)
    device = pred.device
    x = torch.linspace(0, 1, W, device=device).view(1,1,W).expand(1,H,W)
    y = torch.linspace(0, 1, H, device=device).view(1,H,1).expand(1,H,W)
    f_val = 0.1*(torch.sin(2*np.pi*(x+y)) + torch.cos(2*np.pi*(x+y)))  # (1, H, W)
    
    loss_physics = 0.0
    count = 0
    # Loop over time indices to compute time derivative using forward difference.
    for t in range(T_out-1):
        # Temporal derivative: (w(t+1) - w(t))/dt
        w_t = (pred[:, t+1, :, :] - pred[:, t, :, :]) / dt  # (batch, H, W)
        
        # Spatial derivatives using periodic finite differences
        w = pred[:, t, :, :]  # (batch, H, W)
        # x-derivative:
        w_roll_right = torch.roll(w, shifts=-1, dims=2)
        w_roll_left = torch.roll(w, shifts=1, dims=2)
        w_x = (w_roll_right - w_roll_left) / (2*dx)
        
        # y-derivative:
        w_roll_down = torch.roll(w, shifts=-1, dims=1)
        w_roll_up = torch.roll(w, shifts=1, dims=1)
        w_y = (w_roll_down - w_roll_up) / (2*dy)
        
        # Second order derivatives for Laplacian
        w_xx = (w_roll_right - 2*w + w_roll_left) / (dx**2)
        w_yy = (w_roll_down - 2*w + w_roll_up) / (dy**2)
        lap = w_xx + w_yy
        
        # Compute velocity (u, v) from w using FFT-based method (differentiable)
        u, v = compute_velocity_torch(w)  # each: (batch, H, W)
        
        adv = u * w_x + v * w_y  # (batch, H, W)
        
        # PDE residual: r = ∂ₜw + adv - ν * lap - f
        r = w_t + adv - nu * lap - f_val  # broadcast f_val over batch
        
        loss_physics += torch.mean(r**2)
        count += 1
    loss_physics = loss_physics / count
    return loss_physics


In [ ]:
# =============================================================================
# 4. 학습 루프 및 시각화 (Physics-Informed Loss 포함)
# =============================================================================
def train_model_3d(model, train_loader, test_loader, epochs=300, lambda_phys=1.0, dt_out=1.0):
    """
    lambda_phys: physics loss에 곱할 가중치.
    dt_out: 출력 시간 간격 (예: 타겟 시간이 정수일 경우 1.0)
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)
    loss_fn = nn.MSELoss()
    train_losses = []
    test_losses = []
    
    # 공간 간격 (도메인이 [0,1])
    # H, W는 dataset의 target에서 가져올 수 있음; 여기서는 64x64로 가정
    H = 64
    W = 64
    dx = 1.0 / W
    dy = 1.0 / H
    
    for epoch in range(1, epochs+1):
        model.train()
        total_loss = 0.0
        for input_data, target in train_loader:
            # input_data: (batch, T_in, H, W, 4), target: (batch, T_out, H, W, 1)
            input_data = input_data.to(device)
            target = target.to(device)
            optimizer.zero_grad()
            output = model(input_data)  # (batch, T_out, H, W, 1)
            loss_data = loss_fn(output, target)
            loss_phys = compute_physics_loss(output, nu, dt_out, dx, dy)
            loss = loss_data + lambda_phys * loss_phys
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * input_data.size(0)
        total_loss /= len(train_loader.dataset)
        train_losses.append(total_loss)
        
        model.eval()
        total_loss_test = 0.0
        with torch.no_grad():
            for input_data, target in test_loader:
                input_data = input_data.to(device)
                target = target.to(device)
                output = model(input_data)
                loss_data = loss_fn(output, target)
                loss_phys = compute_physics_loss(output, nu, dt_out, dx, dy)
                loss = loss_data + lambda_phys * loss_phys
                total_loss_test += loss.item() * input_data.size(0)
        total_loss_test /= len(test_loader.dataset)
        test_losses.append(total_loss_test)
        scheduler.step()
        print(f"Epoch {epoch:03d}: Train Loss = {total_loss:.6f}, Test Loss = {total_loss_test:.6f}")
    
        # Optionally plot loss curves every few epochs
    plt.figure(figsize=(8,6))
    plt.plot(train_losses, label="Train Loss")
    plt.plot(test_losses, label="Test Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Total Loss")
    plt.legend()
    plt.title("Training and Test Loss Curves (FNO3d with Physics Loss)")
    plt.show()
    
    return train_losses, test_losses

def animate_continuous_motion(model, test_loader, num_interp=10, save_path='continuous_motion.gif'):
    """
    테스트 데이터셋의 첫 번째 샘플에 대해, ground truth와 FNO 모델의 예측 결과의 
    시간에 따른 변화를 연속 애니메이션으로 생성하여 .gif 파일로 저장합니다.
    
    model 출력과 타겟 텐서의 shape은 (batch, T_out, H, W, 1)이어야 합니다.
    num_interp: 인접 스냅샷 사이에 생성할 보간 프레임 수.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    with torch.no_grad():
        input_data, target = next(iter(test_loader))
        input_data = input_data.to(device)
        pred = model(input_data)  # (batch, T_out, H, W, 1)
    
    # 첫 번째 샘플 선택 및 squeeze -> (T_out, H, W)
    sample_pred = pred[0].cpu().numpy().squeeze(-1)
    sample_target = target[0].cpu().numpy().squeeze(-1)
    T_out = sample_pred.shape[0]
    
    # 보간 함수: 인접 프레임 사이에 num_interp개의 중간 프레임 생성
    def interpolate_frames(frames, num_interp):
        T, H, W = frames.shape
        interp_frames = []
        for i in range(T - 1):
            for alpha in np.linspace(0, 1, num_interp, endpoint=False):
                interp_frames.append((1 - alpha) * frames[i] + alpha * frames[i+1])
        interp_frames.append(frames[-1])
        return np.array(interp_frames)
    
    interp_target = interpolate_frames(sample_target, num_interp)  # (new_T, H, W)
    interp_pred = interpolate_frames(sample_pred, num_interp)
    total_frames = interp_target.shape[0]
    
    # Figure와 두 개의 서브플롯 (좌측: Ground Truth, 우측: Prediction)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    im1 = ax1.imshow(interp_target[0], cmap="RdBu", origin="lower")
    ax1.set_title("Ground Truth (frame 1)")
    plt.colorbar(im1, ax=ax1)
    
    im2 = ax2.imshow(interp_pred[0], cmap="RdBu", origin="lower")
    ax2.set_title("Prediction (frame 1)")
    plt.colorbar(im2, ax=ax2)
    
    def update(frame):
        im1.set_data(interp_target[frame])
        im2.set_data(interp_pred[frame])
        ax1.set_title(f"Ground Truth (frame {frame+1}/{total_frames})")
        ax2.set_title(f"Prediction (frame {frame+1}/{total_frames})")
        return [im1, im2]
    
    ani = animation.FuncAnimation(fig, update, frames=total_frames, interval=100, blit=True)
    writer = animation.PillowWriter(fps=10)
    ani.save(save_path, writer=writer)
    print("Animation saved as", save_path)
    plt.close(fig)
    return ani


In [ ]:

# =============================================================================
# 5. 전체 파이프라인 실행
# =============================================================================
if __name__ == '__main__':
    # PDE simulation parameters
    N = 64         # spatial resolution: 64x64
    dt = 0.005     # 시간 스텝 (안정성을 위해 충분히 작게)
    t_final = 50   # 최종 시각
    nu = 1e-3      # 점성 계수 (예시)
    
    # 데이터셋에서 사용할 시간 구간
    T_in_range = (0, 10)          # 입력: t=0 ~ 10 (정수 시간 스냅샷)
    T_out_times = [20, 30, 40, 49] # 타겟: 예시로 t = 20,30,40,49
    
    num_train = 20   # 시연을 위한 샘플 수 (실제 연구에서는 더 많은 샘플 사용)
    num_test = 4
    
    # Navier–Stokes PDE solver를 통해 데이터 생성
    dataset_solver_train = NavierStokesDatasetFromSolver(num_samples=num_train, N=N, dt=dt, t_final=t_final, nu=nu, 
                                                          T_in_range=T_in_range, T_out_times=T_out_times)
    dataset_solver_test  = NavierStokesDatasetFromSolver(num_samples=num_test, N=N, dt=dt, t_final=t_final, nu=nu, 
                                                          T_in_range=T_in_range, T_out_times=T_out_times)
    
    train_loader = DataLoader(dataset_solver_train, batch_size=2, shuffle=True)
    test_loader = DataLoader(dataset_solver_test, batch_size=2, shuffle=False)
    
    # 모델 하이퍼파라미터: Fourier 모드 수 (시간: 4, x: 16, y: 16), width: 32, T_in: 11, T_out: len(T_out_times)
    modes_t, modes_x, modes_y = 4, 16, 16
    width = 32
    T_out = len(T_out_times)
    T_in = len(np.arange(T_in_range[0], T_in_range[1]+1))  # 예: 0~10이면 11
    model = FNO3d(modes_t, modes_x, modes_y, width, T_in, T_out)
    
    # 학습 (physics-informed loss 포함)
    train_losses, test_losses = train_model_3d(model, train_loader, test_loader, epochs=500, lambda_phys=1.0, dt_out=1.0)
    
    # 연속적인 animation 생성 및 .gif 파일 저장
    ani = animate_continuous_motion(model, test_loader, num_interp=10, save_path='continuous_motion.gif')